In [11]:
# Scan all source files and build full scan file
import datetime
import pandas as pd
import numpy as np
import os
from collections import defaultdict
import dask.dataframe as dd
import sys

In [12]:
s3_monitor_dir = '/Users/brianm/Dropbox/cc_AuroraInsight/data/s3/s3_monitor/'
s3_results_dir = '/Users/brianm/Dropbox/cc_AuroraInsight/data/s3'

s3_results_file = 's3_scan6.csv'
s3_results_file_path = os.path.join(s3_results_dir, s3_results_file)

In [13]:
file_list = os.listdir(s3_monitor_dir)
file_list_2 = os.listdir(os.path.join(s3_monitor_dir, 's3_monitor'))
file_list.remove('s3_monitor')

In [14]:
len(file_list)

412470

In [5]:
file_list = [name for name in file_list 
             if '.swp' not in name and 
             os.path.isfile(os.path.join(s3_monitor_dir, name))]


In [6]:
len(file_list)

412470

In [1]:
s3_monitor_lists = []




In [2]:
columns = ['id', 'bucket', 'time', 'time_ms?','accessw_ip', 
           'arn', 'full_action', 'key', 'basic_action', 'key_extended', 
           'http', 'response', 'size', 'software_method',
           'token', 'sig_version',  'encoding', 'header', 's3_location',
           'aws_version'
          ]
needed_columns =  ['bucket', 'time', 'accessw_ip', 
          'full_action', 'key', 'basic_action', 'size', 
            'software_method'
                  ]



In [3]:
num_columns = 27

In [4]:
columns = ['col_{}'.format(x) for x in range(num_columns) ]

In [5]:
drop_columns = [
    'col_0',  
    'col_6',
    'col_11',
    'col_14',
    'col_15',
    'col_16',
    'col_19', 'col_20', 'col_21', 'col_22', 'col_23', 'col_24', ]

In [6]:
new_columns = [
    's3_bucket',
    'datetime',
    'ms',
    'source_ip',
    'arn',
    'operation',
    's3_key',
    'op_key',
    'http_resp',
    'part_size',
    'size',
    'sw_used',
    'unknown_1',
    'source_file',
    'source_dir'    
]

In [7]:
save_df = None

In [8]:
def process_df(this_df):
    global save_df
    this_df['col_2'] = this_df['col_2'].map(lambda x: x.replace('[',''))
    this_df['col_2'] = this_df['col_2'].map(lambda x: datetime.datetime.strptime(x,'%d/%b/%Y:%H:%M:%S'))
    this_df['col_3'] = this_df['col_3'].map(lambda x: x.replace(']',''))
    this_df = this_df.drop(drop_columns, axis=1)
    this_df.columns = new_columns
    save_df = this_df
    #print(this_df)
    this_df['s3_key'] = this_df['s3_key'].apply(str)
    this_df['prefix'] = this_df['s3_key'].map(lambda x: "/".join(x.split('/')[:-1]))
    this_df['file'] = this_df['s3_key'].map(lambda x: x.split('/')[-1])
    save_df = this_df
    if '-' in list(this_df['size'].values):
        this_df['size'] = this_df['size'].map(lambda x: -1 if '-' in str(x) else x)
    if '-' in list(this_df['part_size'].values):
        this_df['part_size'] = this_df['part_size'].map(lambda x: -1 if '-' in str(x) else x)
      
    return this_df

In [9]:
length_counts = defaultdict(int)
first_time = True
for file_name in file_list:
    df_content = pd.read_csv(
        os.path.join(s3_monitor_dir, file_name),
        delimiter=' ',
        quotechar='"',
        header=None )
    #print(file_name)
    df_content.columns = columns[:25]
    length_counts[len(df_content.columns)] += 1
    df_content['source_file'] = file_name
    df_content['source_dir'] = "."
    df_content = process_df(df_content.copy())
    num_columns = len(df_content.columns)
    if first_time:
        first_time = False
        df_content.to_csv(s3_results_file_path, mode='w', header=True, index=True)               
    df_content.to_csv(s3_results_file_path, mode='a', header=False, index=True)


    



  
    
    

NameError: name 'defaultdict' is not defined

In [10]:
save_df

In [29]:

for file_name in file_list_2:
    df_content = pd.read_csv(
        os.path.join(os.path.join(s3_monitor_dir,'s3_monitor'), file_name),
        delimiter=' ',
        quotechar='"')
    #print(file_name)
    df_content.columns = columns[:25]
    length_counts[len(df_content.columns)] += 1
    df_content['source_file'] = file_name
    df_content['source_dir'] = "./s3_monitor/"
    df_content = process_df(df_content)
    num_columns = len(df_content.columns)
    df_content.to_csv(s3_results_file_path, mode='a', header=False, index=True)

    